In [3]:
using ODBC, DataFrames


ODBC.setunixODBC()

conn = ODBC.Connection("telecom")

ODBC.Connection(telecom)

In [5]:
sql = """
SELECT
    *
FROM clean_CDR
WHERE 
    calling_nbr in (SELECT client_nbr FROM clean_user_info) AND
    called_nbr in (SELECT client_nbr FROM clean_user_info)
"""

# network
clean_cdr = DBInterface.execute(conn, sql) |> DataFrame

Row,time,day_of_week,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,cell_id
,Int64?,Int32?,String?,String?,Int32?,String?,String?,String?
1,9,5,284y280u,0838,36,0838,jgos42b6,355D
2,9,5,27jql2yi,0838,36,0838,2jnq2ss2,3A3C
3,14,5,27ko3a52,0838,26,0838,3u008khu,39F6
4,17,5,29mykvh2,0838,22,0838,2bek42dq,384E
5,18,5,29mykvh2,0838,123,0838,2bek42dq,384E
6,19,5,27mpjcrm,0838,17,0838,gwog8kf8,37D5
7,19,5,27mpjcrm,0838,1,0838,gwog8kf8,37D5
8,19,5,27mpjcrm,0838,18,0838,gwog8kf8,37D5
9,21,5,29mykvh2,0838,19,0838,2bek42dq,39B0


In [6]:
network = deepcopy(clean_cdr)
network[1:3, :]

Row,time,day_of_week,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,cell_id
,Int64?,Int32?,String?,String?,Int32?,String?,String?,String?
1,9,5,284y280u,0838,36,0838,jgos42b6,355D
2,9,5,27jql2yi,0838,36,0838,2jnq2ss2,3A3C
3,14,5,27ko3a52,0838,26,0838,3u008khu,39F6


In [12]:
using Distributions

idx = sample(1:size(network, 1), 80_000; replace = false)
network_sample = network[idx, :]

Row,time,day_of_week,calling_nbr,calling_area_code,duration,called_area_code,called_nbr,cell_id
,Int64?,Int32?,String?,String?,Int32?,String?,String?,String?
1,14,7,g9hk3mq0,0838,82,0838,lr9tjdgv,3CB9
2,12,7,29o1jcue,0838,22,0838,eorv3a57,3B17
3,15,7,6zfzjcug,0838,24,0838,eh7xjd8f,352A
4,20,7,lxp43q4b,0838,34,0838,mspu331r,3647
5,12,2,njo44x93,0838,55,0838,27ng3aoi,35C0
6,9,2,b45054t6,0838,20,0838,d6on32kz,3AFB
7,16,5,6tnw5zd4,0838,379,0838,2olukfqe,398E
8,22,4,jcuw3a82,0838,12,0838,fm6c5zis,35C3
9,9,7,9tk3f0lh,0838,96,0838,5bf45k33,37BB


In [15]:
# using Pkg; Pkg.add("Parquet2")
using Parquet

[ Info: Precompiling CategoricalArraysStructTypesExt [e0b1e8f7-f9d4-55ed-aa4c-3cb4a66b5cfe]
[ Info: Precompiling CategoricalArraysJSONExt [3659b7b4-73bc-5a91-ba4f-cc4948098fdb]


In [18]:
write_parquet("../../../data/processed/network_sample.parquet", network_sample)

4

In [19]:
write_parquet("../../../data/processed/network.parquet", network)

4

In [20]:
sql = """
SELECT
    *
FROM clean_user_info
"""
user_info = DBInterface.execute(conn, sql) |> DataFrame

Row,client_nbr,register_district,age,phone_brand,phone_level,phone_price,evdo_support_flag,mou_local,mou_dist,network_vol,evdo_use_flag,onex_use_flag,evdo_vol,onex_vol,arpu,e9_service_flag,e6_service_flag,e9_service_premium_flag,8card_service_flag,smart_phone_flag,card_phone_flag,rural_flag,employ_flag,student_flag
,String?,Int32,Int64?,String?,Int32,Int32?,Int64?,Float32?,Float32?,Float32?,String?,String?,Float32?,Float32?,Float32?,Int64?,Int64?,Int64?,Int32,Int32,Int32,Int32,Int32,Int32
1,6m6s3n3s,1,32,SAMSUNG,5,4890,1,176.08,37.55,256.16,1,1,255.07,0.03,0.0,1,0,0,0,1,0,0,1,0
2,6twk3n3s,1,32,中兴,3,899,1,28.75,0.43,45.54,1,0,45.54,0.0,0.1,1,0,0,0,1,0,0,1,0
3,mm7pjcxf,1,66,华立时代,2,330,0,11.5,0.0,0.0,0,0,0.0,0.0,8.0,0,0,0,0,0,0,0,1,0
4,fq5m3q1g,2,31,SAMSUNG,1,299,0,233.58,0.25,0.0,0,0,0.0,0.0,38.0,0,0,0,0,0,0,0,1,0
5,4kkhjsib,1,50,SAMSUNG,1,199,0,13.92,0.0,0.0,0,0,0.0,0.0,0.0,1,0,0,0,0,0,1,1,0
6,iltc4wv5,1,39,华为,3,1090,1,11.9,0.0,911.45,1,0,911.45,0.0,6.2,1,0,0,0,1,0,0,1,0
7,efqbk0gj,4,54,世纪星宇,2,399,1,38.68,0.0,0.0,0,0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,0
8,bvdlk0dq,1,39,华为,3,838,1,40.98,23.43,86.5,1,0,86.5,0.0,66.56,1,0,0,0,1,0,1,1,0
9,hbvgl0f1,4,31,中兴,1,199,0,31.03,16.15,0.0,0,0,0.0,0.0,6.0,1,0,0,0,0,0,1,1,0


In [21]:
write_parquet("../../../data/processed/user_info.parquet", user_info)

4